## Group 4 - Lab 6 - Python

Katiuska Olivera Quevedo (20172533) <br/> 
Rosemery Fernandez Sanchez (20172635) <br/> 
Aliro Cabrera Florez (20152034) <br/> 
Jose Uscamayta Quispe (20195674) <br/> 

In [1]:
import pandas as pd
import numpy as np
import pyreadr
from sklearn import preprocessing
import patsy


In [2]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

In [3]:
import hdmpy
import numpy as np
import random
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors

In [4]:
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, ElasticNetCV
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import itertools
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_categorical_dtype
from itertools import compress
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.feature_selection import SelectFromModel

### DATA 

In [14]:
growth = pyreadr.read_r("C:/Users/ok235/OneDrive/Documentos/GitHub/ECO224/Labs/data/GrowthData.RData") 


In [15]:
data1 = growth[ 'GrowthData' ]
list(data1)


['Outcome',
 'intercept',
 'gdpsh465',
 'bmp1l',
 'freeop',
 'freetar',
 'h65',
 'hm65',
 'hf65',
 'p65',
 'pm65',
 'pf65',
 's65',
 'sm65',
 'sf65',
 'fert65',
 'mort65',
 'lifee065',
 'gpop1',
 'fert1',
 'mort1',
 'invsh41',
 'geetot1',
 'geerec1',
 'gde1',
 'govwb1',
 'govsh41',
 'gvxdxe41',
 'high65',
 'highm65',
 'highf65',
 'highc65',
 'highcm65',
 'highcf65',
 'human65',
 'humanm65',
 'humanf65',
 'hyr65',
 'hyrm65',
 'hyrf65',
 'no65',
 'nom65',
 'nof65',
 'pinstab1',
 'pop65',
 'worker65',
 'pop1565',
 'pop6565',
 'sec65',
 'secm65',
 'secf65',
 'secc65',
 'seccm65',
 'seccf65',
 'syr65',
 'syrm65',
 'syrf65',
 'teapri65',
 'teasec65',
 'ex1',
 'im1',
 'xr65',
 'tot1']

In [17]:
data1.shape

(90, 63)

### We create the main variables

y = outcome: growth rate <br/>
d = treatment: initial wealth <br/>
x = controls: country characteristics <br/>

In [21]:
y = data1['Outcome']
d = data1['gdpsh465']
x = data1.drop(['Outcome','gdpsh465'], 1)

### 1. OLS without including the country characteristics

After preprocessing the data, we first look at simple regression of $Y_{j,t}$ on $D_{j,t-1}$ without controls as a baseline model.

In [23]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

In [25]:
reg_ols  = sm.OLS(y, d).fit()
print(reg_ols.summary())

                                 OLS Regression Results                                
Dep. Variable:                Outcome   R-squared (uncentered):                   0.438
Model:                            OLS   Adj. R-squared (uncentered):              0.432
Method:                 Least Squares   F-statistic:                              69.38
Date:                Sat, 27 Nov 2021   Prob (F-statistic):                    9.13e-13
Time:                        00:49:59   Log-Likelihood:                          139.82
No. Observations:                  90   AIC:                                     -277.6
Df Residuals:                      89   BIC:                                     -275.1
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [86]:
coef1 = reg_ols.summary2().tables[1]['Coef.']['gdpsh465']
err1 =reg_ols.summary2().tables[1]['Std.Err.']['gdpsh465']

The point estimate is $0.0058$ with the confidence interval ranging from 0.004 to 0.007. This suggests that increases in the country outcome are very low related to the initial wealth - if the intial wealth increase by 1% relative to a trend then the predicted outcome goes up by 0.005%, without controlling for countries' characteristics.

### 2. OLS including the country characteristics

Now we're gonna run an OLS regression with the controls.

In [32]:
model = "y ~ d + x"
baseline_ols = smf.ols(model , data=data1).fit()
print(baseline_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.887
Model:                            OLS   Adj. R-squared:                  0.641
Method:                 Least Squares   F-statistic:                     3.607
Date:                Sat, 27 Nov 2021   Prob (F-statistic):           0.000200
Time:                        01:02:21   Log-Likelihood:                 238.24
No. Observations:                  90   AIC:                            -352.5
Df Residuals:                      28   BIC:                            -197.5
Df Model:                          61                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1236      0.392      0.315      0.7

In [95]:
coef2= baseline_ols.summary2().tables[1]['Coef.']['Intercept']
err2=baseline_ols.summary2().tables[1]['Std.Err.']['Intercept']

The point estimate is $0.1236$, with strong multicollinearity problems. This suggests that increases in the country outcome are  related to the initial wealth - if the intial wealth increase by 1% relative to a trend then the predicted outcome goes up by 0.12%, with control for countries' characteristics.

### Setting DML

In [38]:
def DML2_for_PLM(x, d, y, dreg, yreg, nfold):
    
    # Num ob observations
    nobs = x.shape[0]
    
    # Define folds indices 
    list_1 = [*range(0, nfold, 1)]*nobs
    sample = np.random.choice(nobs,nobs, replace=False).tolist()
    foldid = [list_1[index] for index in sample]

    # Create split function(similar to R)
    def split(x, f):
        count = max(f) + 1
        return tuple( list(itertools.compress(x, (el == i for el in f))) for i in range(count) ) 

    # Split observation indices into folds 
    list_2 = [*range(0, nobs, 1)]
    I = split(list_2, foldid)
    
    # Create array to save errors 
    dtil = np.zeros( len(x) ).reshape( len(x) , 1 )
    ytil = np.zeros( len(x) ).reshape( len(x) , 1 )
    
    # loop to save results
    for b in range(0,len(I)):
    
        # Split data - index to keep are in mask as booleans
        include_idx = set(I[b])  #Here should go I[b] Set is more efficient, but doesn't reorder your elements if that is desireable
        mask = np.array([(i in include_idx) for i in range(len(x))])

        # Lasso regression, excluding folds selected 
        dfit = dreg(x[~mask,], d[~mask,])
        yfit = yreg(x[~mask,], y[~mask,])

        # predict estimates using the 
        dhat = dfit.predict( x[mask,] )
        yhat = yfit.predict( x[mask,] )

        # save errors  
        dtil[mask] =  d[mask,] - dhat.reshape( len(I[b]) , 1 )
        ytil[mask] = y[mask,] - yhat.reshape( len(I[b]) , 1 )
        print(b, " ")
    
    # Create dataframe 
    data_2 = pd.DataFrame(np.concatenate( ( ytil, dtil ), axis = 1), columns = ['ytil','dtil'])
   
    # OLS clustering at the County level
    model = "ytil ~ dtil"
    baseline_ols = smf.ols(model , data = data_2 ).fit()
    coef_est = baseline_ols.summary2().tables[1]['Coef.']['dtil']
    se = baseline_ols.summary2().tables[1]['Std.Err.']['dtil']
    
    Final_result = { 'coef_est' : coef_est , 'se' : se , 'dtil' : dtil , 'ytil' : ytil }

    print("Coefficient is {}, SE is equal to {}".format(coef_est, se))
    
    return Final_result

In [35]:
y = y.to_numpy().reshape( len(y) , 1 )
d = d.to_numpy().reshape( len(y) , 1 )
x = x.to_numpy()

### 3. DML using Lasso to predict y an d

In [39]:
def dreg(x,d):
    alpha=0.00000001
    result = linear_model.Lasso(alpha = alpha).fit(x, d)
    return result

def yreg(x,y):
    alpha=0.00000001
    result = linear_model.Lasso(alpha = alpha).fit(x, y)
    return result

DML2_lasso = DML2_for_PLM(x, d, y, dreg, yreg, 10)

C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.7251612094835438, tolerance: 0.006618842106369448
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.012799180587603314, tolerance: 2.05523125087365e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.9009933676748404, tolerance: 0.0060111571558129516
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: Co

0  
1  
2  
3  
4  


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.7226552514491494, tolerance: 0.006512132190847078
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.01373620086003732, tolerance: 1.7063584905759185e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.7423139639451742, tolerance: 0.0064163684077558816
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: C

5  
6  
7  
8  
9  
Coefficient is 0.006798272022925064, SE is equal to 0.011643718233111295


The point estimate is $0.006$. This suggests that increases in the country outcome are very low related to the initial wealth (similar tu our first OLS) - if the intial wealth increase by 1% relative to a trend then the predicted outcome goes up by 0.006%, with control for countries' characteristics.

### 4. DML using Post-Lasso to predict y an d.

In [40]:
class Lasso_post:
    
    def __init__(self, alpha ):
        self.alpha = alpha

        
    def fit( self, X, Y ):
        self.X = X
        self.Y = Y
        lasso = linear_model.Lasso( alpha = self.alpha ).fit( X , Y )
        model = SelectFromModel( lasso , prefit = True )
        X_new = model.transform( X )
        # Gettin indices from columns which has variance for regression
        index_X = model.get_support()
        
        self.index = index_X
        new_x = X[ : ,  index_X ]
        
        lasso2 = linear_model.Lasso( alpha = self.alpha ).fit( new_x , Y )
        self.model = lasso2
        
        return self
    
    def predict( self , X ):
        
        dropped_X = X[ : , self.index ]
        
        predictions = self.model.predict( dropped_X )
        
        return predictions

In [41]:
def dreg(x,d):
    alpha=0.00000001
    result = Lasso_post( alpha = alpha ).fit( x , d )
    return result

def yreg( x , y ):
    alpha = 0.00000001
    result = Lasso_post( alpha = alpha ).fit( x , y )
    return result

DML2_lasso_post = DML2_for_PLM(x, d, y, dreg, yreg, 10)

C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.7434336906736052, tolerance: 0.0063246947641623094
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.7592612861985834, tolerance: 0.0063246947641623094
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.012494605975976777, tolerance: 1.6121787541581912e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530:

0  
1  
2  


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.014031949355075686, tolerance: 2.264824814728605e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.6910978438895217, tolerance: 0.006779131613543115
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.6979989635710823, tolerance: 0.006779131613543115
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: Co

3  
4  
5  


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.014711882557359324, tolerance: 2.2004437187142165e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.015152295466090093, tolerance: 2.2004437187142165e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.854515857589303, tolerance: 0.006483883551645648
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530

6  
7  
8  
9  
Coefficient is 0.03329427369670361, SE is equal to 0.011766093101544658


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.8596667743352536, tolerance: 0.006325295327288889
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.8785561617823481, tolerance: 0.006325295327288889
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.016177356279149128, tolerance: 2.115992165254656e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: Co

In [42]:
import hdmpy
from statsmodels.tools import add_constant

In [43]:
class rlasso_hdmy:
    
    def __init__(self, post ):
        self.post = post
       
    def fit( self, X, Y ):
        
        self.X = X
        self.Y = Y
        
        # Standarization of X and Y
        self.rlasso_model = hdmpy.rlasso( X , Y , post = self.post )                
        return self
    
    def predict( self , X_1 ):
        self.X_1 = X_1
        beta = self.rlasso_model.est['coefficients'].to_numpy()
        
        if beta.sum() == 0:
            prediction = np.repeat( self.rlasso_model.est['intercept'] , self.X_1.shape[0] )
        
        else:
            prediction = ( add_constant( self.X_1 , has_constant = 'add') @ beta ).flatten()
                
        return prediction

In [44]:
# Post = false
def dreg(x, d):
    result = rlasso_hdmy( post = False ).fit( x , d )
    return result

def yreg(x,y):
    result = rlasso_hdmy( post = False ).fit( x , y )
    return result

DML2_lasso_hdmpy = DML2_for_PLM(x, d, y, dreg, yreg, 10)

C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_ba

0  


C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_ba

1  


C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_ba

...

7  


C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_ba

8  


C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
C:\Users\ok235\anaconda3\lib\site-packages\numpy\lib\function_ba

9  
Coefficient is -0.037151840583525296, SE is equal to 0.01448374147317848


The point estimate is  -0.0372 . This suggests that decreases in the country outcome negative related to the initial wealth - if the intial wealth increase by 1% relative to a trend then the predicted outcome goes down by -0.037%.

### 5. DML using Elastic Net to predict y an d

We decided to use StandardScaler. We proved this in the commented code below. It normalizes X by subtracting the mean and dividing by the l2-norm

In [45]:
class standard_skl_model:
    
    def __init__(self, model ):
        self.model = model
       
    def fit( self, X, Y ):
        
        # Standarization of X and Y
        self.scaler_X = StandardScaler()
        self.scaler_X.fit( X )
        std_X = self.scaler_X.transform( X )
                
        self.model.fit( std_X , Y )
                
        return self
    
    def predict( self , X ):
        
        self.scaler_X = StandardScaler()
        self.scaler_X.fit( X )
        std_X = self.scaler_X.transform( X )
        
        prediction = self.model.predict( std_X )
        
        return prediction

In [48]:
def dreg(x,d):
    result = standard_skl_model( ElasticNetCV( cv = 10 , random_state = 0 , l1_ratio = 0.5, max_iter = 100000 ) ).fit( x, d )
    return result

def yreg(x,y):
    result = standard_skl_model( ElasticNetCV( cv = 10 , random_state = 0 , l1_ratio = 0.5, max_iter = 100000 ) ).fit( x, y )
    return result

DML2_elnet = DML2_for_PLM(x, d, y, dreg, yreg, 10)

C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0  


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


1  


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


...

7  


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


8  


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


9  
Coefficient is 0.008188248172608341, SE is equal to 0.012136628007422266


The point estimate is  0.00818. This suggests that increases in the country outcome are very low related to the initial wealth - if the intial wealth increase by 1% relative to a trend then the predicted outcome goes up by 0.008%.

### 6.DML using Ridge to predict y an d

In [49]:
def dreg(x,d):
    result = standard_skl_model( ElasticNetCV( cv = 10 ,  random_state = 0 , l1_ratio = 0.0001 ) ).fit( x, d )
    return result

def yreg(x,y):
    result = standard_skl_model( ElasticNetCV( cv = 10 , random_state = 0 , l1_ratio = 0.0001 ) ).fit( x, y )
    return result

DML2_ridge = DML2_for_PLM(x, d, y, dreg, yreg, 10)

C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0  


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


1  


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


...

7  


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


8  


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


9  
Coefficient is -0.015659275084224086, SE is equal to 0.009844583994330549


The point estimate is -0.01565 . This suggests that decreases in the country outcome negative related to the initial wealth - if the intial wealth increase by 1% relative to a trend then the predicted outcome goes down by -0.016%

### 7. DML using Random Forest to predict y an d.

In [50]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [55]:
def dreg(x,d):
    result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 61 , n_jobs = 4 , min_samples_leaf = 5 ).fit( x, d )
    return result

def yreg(x,y):
    result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 61 , n_jobs = 4 , min_samples_leaf = 5 ).fit( x, y )
    return result

DML2_RF = DML2_for_PLM(x, d, y, dreg, yreg, 10) 

<ipython-input-55-222fcda8970a>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 61 , n_jobs = 4 , min_samples_leaf = 5 ).fit( x, d )
<ipython-input-55-222fcda8970a>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 61 , n_jobs = 4 , min_samples_leaf = 5 ).fit( x, y )


0  


<ipython-input-55-222fcda8970a>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 61 , n_jobs = 4 , min_samples_leaf = 5 ).fit( x, d )
<ipython-input-55-222fcda8970a>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 61 , n_jobs = 4 , min_samples_leaf = 5 ).fit( x, y )


1  


<ipython-input-55-222fcda8970a>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 61 , n_jobs = 4 , min_samples_leaf = 5 ).fit( x, d )
<ipython-input-55-222fcda8970a>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 61 , n_jobs = 4 , min_samples_leaf = 5 ).fit( x, y )


...

7  


<ipython-input-55-222fcda8970a>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 61 , n_jobs = 4 , min_samples_leaf = 5 ).fit( x, d )
<ipython-input-55-222fcda8970a>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 61 , n_jobs = 4 , min_samples_leaf = 5 ).fit( x, y )


8  


<ipython-input-55-222fcda8970a>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 61 , n_jobs = 4 , min_samples_leaf = 5 ).fit( x, d )
<ipython-input-55-222fcda8970a>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 61 , n_jobs = 4 , min_samples_leaf = 5 ).fit( x, y )


9  
Coefficient is -0.03346767475642443, SE is equal to 0.011467636423300483


The point estimate is -0.03346 . This suggests that decreases in the country outcome negative related to the initial wealth - if the intial wealth increase by 1% relative to a trend then the predicted outcome goes down by -0.03%

In [59]:
mods = [DML2_lasso, DML2_lasso_post , DML2_ridge, DML2_elnet, DML2_RF]
mods_name = ["DML2_lasso", "DML2_lasso_post", 'DML2_ridge', 'DML2_elnet', 'DML2_RF']

def mdl( model , model_name ):
    
    RMSEY = np.sqrt( np.mean( model[ 'ytil' ] ) ** 2 ) # I have some doubts about these equations...we have to recheck
    RMSED = np.sqrt( np.mean( model[ 'dtil' ] ) ** 2 ) # I have some doubts about these equations...we have to recheck
    
    result = pd.DataFrame( { model_name : [ RMSEY , RMSED ]} , index = [ 'RMSEY' , 'RMSED' ])
    return result

RES = [ mdl( model , name ) for model, name in zip( mods , mods_name ) ]
    

pr_Res = pd.concat( RES, axis = 1)

pr_Res

,DML2_lasso,DML2_lasso_post,DML2_ridge,DML2_elnet,DML2_RF
RMSEY,0.000888,0.002018,1.526557e-17,6.013708e-17,0.001316
RMSED,0.007567,0.007368,4.736952e-16,1.648064e-15,0.018543


This verfies that the function DML2_for_PLM has no errors

In [57]:
np.where(DML2_lasso_post[ 'ytil' ] == 0)[0].size

0

### 8. Run the best method i.e. the best combination of methods to predict y an d

It looks like the best method for predicting Y is CV Ridge, and the best method for predicting D is CV Ridge as well

In [61]:
def dreg(x,d):
    result = standard_skl_model(  ElasticNetCV(cv = 10 , random_state = 0 , alphas = [0]) ).fit( x, d )
    return result


def yreg(x,y):
    result = standard_skl_model( ElasticNetCV( cv = 10 ,  random_state = 0 , l1_ratio = 0.0001 ) ).fit( x, y )
    return result

DML2_best = DML2_for_PLM(x, d, y , dreg, yreg, 10)

C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.44694160332505106, tolerance: 0.0059488819540079495
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model =

0  


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.5496486908987066, tolerance: 0.006112426735505871
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = c

1  


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.47717870073073243, tolerance: 0.005964544927562241
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = 

...

7  


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.4419757546435221, tolerance: 0.005713478514456421
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = c

8  


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.46982558539912844, tolerance: 0.006238344909016694
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = 

9  
Coefficient is 0.002261475085439086, SE is equal to 0.0067389121375971165


The point estimate is 0.0022. This suggests that increases in the country outcome are very low related to the initial wealth - if the intial wealth increase by 1% relative to a trend then the predicted outcome goes up by 0.002%.

### Results 

In [97]:
table = np.zeros( ( 8 , 2 ))
table[ 0 , 0] = coef1
table[ 1 , 0] = coef2
table[ 2 , 0] = DML2_lasso['coef_est']
table[ 3 , 0] = DML2_lasso_post['coef_est']
table[ 4 , 0] = DML2_ridge['coef_est']
table[ 5 , 0] = DML2_elnet['coef_est']
table[ 6 , 0] = DML2_RF['coef_est']
table[ 7 , 0] = DML2_best['coef_est']
table[ 0 , 1] = err1
table[ 1 , 1] = err2
table[ 2 , 1] = DML2_lasso['se']
table[ 3 , 1] = DML2_lasso_post['se']
table[ 4 , 1] = DML2_ridge['se']
table[ 5 , 1] = DML2_elnet['se']
table[ 6 , 1] = DML2_RF['se']
table[ 7 , 1] = DML2_best['se']

In [98]:
table = pd.DataFrame( table , index = [ "Baseline OLS", "OLS with controls", "Lasso", \
             "Post-Lasso","CV Elnet", "CV Ridge", "Random Forest", "Best" ] , \
            columns = ["Estimate","Standard Error"] )
table.round( 3 )

,Estimate,Standard Error
Baseline OLS,0.006,0.001
OLS with controls,0.124,0.392
Lasso,0.007,0.012
Post-Lasso,0.033,0.012
CV Elnet,-0.016,0.010
CV Ridge,0.008,0.012
Random Forest,-0.033,0.011
Best,0.002,0.007


In [92]:
table.to_latex()

'\\begin{tabular}{lrr}\n\\toprule\n{} &  Estimate &  Standard Error \\\\\n\\midrule\nBaseline OLS      &  0.005827 &        0.000700 \\\\\nOLS with controls &       NaN &             NaN \\\\\nLasso             &  0.006798 &        0.011644 \\\\\nPost-Lasso        &  0.033294 &        0.011766 \\\\\nCV Elnet          & -0.015659 &        0.009845 \\\\\nCV Ridge          &  0.008188 &        0.012137 \\\\\nRandom Forest     & -0.033468 &        0.011468 \\\\\nBest              &  0.002261 &        0.006739 \\\\\n\\bottomrule\n\\end{tabular}\n'